# Why Use Φ<sub>ML</sub>'s Tensors

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tum-pbs/PhiML/blob/main/docs/Tensors.ipynb)
&nbsp; • &nbsp; [🌐 **Φ<sub>ML</sub>**](https://github.com/tum-pbs/PhiML)
&nbsp; • &nbsp; [📖 **Documentation**](https://tum-pbs.github.io/PhiML/)
&nbsp; • &nbsp; [🔗 **API**](https://tum-pbs.github.io/PhiML/phiml)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/tum-pbs/PhiML/blob/main/docs/Examples.ipynb) [**Examples**](https://tum-pbs.github.io/PhiML/Examples.html)

While you can call many Φ<sub>ML</sub> function directly with native tensors, such as Jax tensors or NumPy arrays, we recommend wrapping them in Φ<sub>ML</sub> tensors.
These provide several benefits over the native tensors and allow you to write easy-to-read, more concise, more explicit, less error-prone code.

For an introduction into tensors and dimensions, check out the [introduction notebook](Introduction.ht).

In [1]:
%%capture
!pip install phiml

## Named Dimensions

All tensor dimensions in Φ<sub>ML</sub> have a name and type which are part of the [tensor shape](Shapes.html).
When creating a Φ<sub>ML</sub> tensor, you specify the names and types of all dimensions.

In [33]:
from phiml import math, wrap, tensor
from phiml.math import channel, batch, spatial, instance, dual  # dimension types

In [7]:
wrap([0, 1, 2], channel('integers'))

(0, 1, 2) along integersᶜ

In [10]:
data = math.random_uniform(batch(examples=2), spatial(x=4, y=3))
data

(examplesᵇ=2, xˢ=4, yˢ=3) 0.453 ± 0.287 (9e-03...9e-01)

With all dims named, Φ<sub>ML</sub> can automatically match dims without requiring you to expand, squeeze or transpose.
Let's add steps to our `data` tensor along `x`.

In [13]:
steps = wrap([0, 1, 2, 3], 'x:s')
steps

(0, 1, 2, 3) along xˢ

In [14]:
math.print(data + steps)

examples=0  
 0.13327174, 1.7961333 , 2.1541028 , 3.7162008 ,
 0.00906819, 1.5718875 , 2.2791338 , 3.7322206 ,
 0.78915596, 1.8791398 , 2.567636  , 3.185347    along (xˢ=4, yˢ=3)
examples=1  
 0.22575481, 1.1442231 , 2.3747668 , 3.3199682 ,
 0.6644003 , 1.4948416 , 2.8507886 , 3.8840666 ,
 0.6648495 , 1.2069794 , 2.2057896 , 3.0330396   along (xˢ=4, yˢ=3)


As you can see, the *n*th column values now lie in [n-1,n].
Φ<sub>ML</sub> automatically expanded `steps` to the shape `(1, steps, 1)` before adding it to `data`.

This feature works with all Φ<sub>ML</sub> functions. You can think of it in the following way:

> Tensors implicitly have all possible dimensions, but their values are constant along those not listed in the shape.

A consequence of the automatic reshaping is that

> The dim order of Φ<sub>ML</sub> tensors is irrelevant.

In fact, you should never count on a specific order or single out a dimension by its index.

Let's look at another example, creating a meshgrid from scratch.

In [17]:
from phiml import stack, arange

stack({
    'x': arange(spatial(x=4)),
    'y': arange(spatial(y=3))
}, channel('vector'), expand_values=True)

(yˢ=3, xˢ=4, vectorᶜ=x,y) 1.250 ± 1.010 (0e+00...3e+00)

The preferred way of referencing dims, e.g. for slicing tensors is by type or name.

In [24]:
data.examples[1].x[1:3].y[0]

(0.207, 0.206) along xˢ

In [28]:
data[{batch: 1, 'x': slice(1, 3), 'y': 0}]

(0.207, 0.206) along xˢ

Many functions will operate on dims of specific types by default, so you don't need to specify the axis.

In [30]:
math.fft(data)  # operates on x and y because they are spatial

(examplesᵇ=2, xˢ=4, yˢ=3) complex64 |...| < 5.813297748565674

Batch dimensions will be ignored by all functions unless explicitly specified. This makes it trivial to vectorize code w/o using something like `vmap`, simply by passing any batched input. An example of this can be seen [here](https://tum-pbs.github.io/PhiFlow/examples/grids/Batched_Smoke.html).

## Shortened Dim Creation

Functions that only require you to specify name and type, but not the size of dims, allow you to write `<name>:<type_letter>` instead of passing a `Shape` object. Single-letter names additionally default to type spatial and item names can be specified in parentheses.

In [31]:
wrap([1, 2, 3], 'x')

(1, 2, 3) along xˢ

In [34]:
tensor([1, 2, 3], 'vector:c')

(1, 2, 3)

In [35]:
stack([1, 2, 3], 'example:b')

(1, 2, 3) along exampleᵇ

## Printing Options

As you can see, Φ<sub>ML</sub> summarizes tensors by default and color-codes the result text.
The Python formatting options let you customize how a tensor is printed, with options being separated by colons.
Here are some examples:

In [10]:
print(f"{data:summary:color:shape:dtype:.5e}")

(examplesᵇ=2, xˢ=4, yˢ=3) float32 5.70418e-01 ± 2.15591e-01 (1.47583e-01...9.55640e-01)


In [17]:
print(f"{data:full:color:shape:dtype:.3f}")

examples=0  
 0.204, 0.148, 0.273, 0.764,
 0.612, 0.512, 0.838, 0.794,
 0.631, 0.954, 0.453, 0.585  along (xˢ=4, yˢ=3)
examples=1  
 0.824, 0.736, 0.382, 0.608,
 0.467, 0.956, 0.452, 0.453,
 0.422, 0.713, 0.467, 0.443  along (xˢ=4, yˢ=3)


In [21]:
print(f"{data:numpy:no-color:no-shape:no-dtype:.2f}")

[[[0.63 0.61 0.20]
  [0.95 0.51 0.15]
  [0.45 0.84 0.27]
  [0.58 0.79 0.76]]

 [[0.42 0.47 0.82]
  [0.71 0.96 0.74]
  [0.47 0.45 0.38]
  [0.44 0.45 0.61]]]


The order of the formatting arguments is not important.
Supported options are:

**Layout:**
The layout determines what is printed and where. The following options are available:

* `summary` Summarizes the values by mean, standard deviation, minimum and maximum value.
* `row` Prints the tensor as a single-line vector.
* `full` Prints all values in the tensors as a multi-line string.
* `numpy` Uses the formatting of NumPy

**Number format**:
You can additionally specify a format string for floating-point numbers like `.3f` or `.2e`.

**Color**:
Use the keywords `color` or `no-color`.
Currently `color` will use ANSI color codes which are supported by most terminals, IDEs as well as Jupyter notebooks.

**Additional tensor information**:
The keywords `shape`, `no-shape`, `dtype` and `no-dtype` can be used to show or hide additional properties of the tensor.


## Wrapping and Unwrapping

You can wrap existing tensors in Φ<sub>ML</sub> tensors using [`wrap()`](phiml/math#phiml.math.wrap) or [`tensor()`](phiml/math#phiml.math.tensor).
While `tensor()` will convert the data to the [default backend](Convert.html), `wrap()` will keep the data as-is.
In either case, you need to specify the dimension names and types when wrapping a native tensor.

In [24]:
math.use('torch')
math.tensor([0, 1, 2], batch('examples'))

(0, 1, 2) along examplesᵇ

To unwrap a tensor, you can use `tensor.native()` or `math.reshaped_native()` for more control over the result shape.
In both cases, the requested dimension order needs to be specified.

In [25]:
data.native('examples,x,y')

array([[[0.6309898 , 0.611722  , 0.2038249 ],
        [0.9543073 , 0.5123106 , 0.14758302],
        [0.45304176, 0.83762187, 0.27338478],
        [0.5846348 , 0.794242  , 0.7644842 ]],

       [[0.42182267, 0.46698892, 0.82417876],
        [0.7127511 , 0.9556401 , 0.7356606 ],
        [0.46661076, 0.45159382, 0.38228187],
        [0.4433869 , 0.45273894, 0.608222  ]]], dtype=float32)

Similarly, you can get the NumPy representation:

In [30]:
data.numpy('examples,x,y')

array([[[0.6309898 , 0.611722  , 0.2038249 ],
        [0.9543073 , 0.5123106 , 0.14758302],
        [0.45304176, 0.83762187, 0.27338478],
        [0.5846348 , 0.794242  , 0.7644842 ]],

       [[0.42182267, 0.46698892, 0.82417876],
        [0.7127511 , 0.9556401 , 0.7356606 ],
        [0.46661076, 0.45159382, 0.38228187],
        [0.4433869 , 0.45273894, 0.608222  ]]], dtype=float32)

## Further Reading

Check out the [examples](https://tum-pbs.github.io/PhiML/Examples.html) to see how using Φ<sub>ML</sub>'s tensors is different from the other libraries.

Learn more about the [dimension types](Shapes.html) and their [advantages](Dimension_Names_Types.html).

Φ<sub>ML</sub> unifies [data types](Data_Types.html) as well and lets you set the floating point precision globally or by context.

While the dimensionality of neural networks must be specified during network creation, this is not the case for math functions.
These [automatically adapt to the number of spatial dimensions of the data that is passed in](N_Dimensional.html).

[🌐 **Φ<sub>ML</sub>**](https://github.com/tum-pbs/PhiML)
&nbsp; • &nbsp; [📖 **Documentation**](https://tum-pbs.github.io/PhiML/)
&nbsp; • &nbsp; [🔗 **API**](https://tum-pbs.github.io/PhiML/phiml)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/tum-pbs/PhiML/blob/main/docs/Examples.ipynb) [**Examples**](https://tum-pbs.github.io/PhiML/Examples.html)